# Logistic Regression Example
## Small Introduction

The general formula of linear regression is familiar to most, and is given by $Y = b_0+b_1 X_1$. Graphically, it represents a linear correlation between two continuous variables ($Y$ and $X_1$). But, what type of formula would we need in order to describe a binary variable?
It turns out that the best formula is logistic regression formula, which, when plotted, will have a sigmoid shape.
Here is a figure showing the distinction between linear and logistic regression:

<img src="https://raw.githubusercontent.com/abdulrahman1123/analysis_examples/main/logistic_reg_intro.png" width=525 height=200 />


The aim of logistic regression modeling is to find the best formula that fits the data the best (i.e. with lowest error possible).

The formula of logistic regression is closely related to that of the linear regression, and is given by:

$$Probability (Y) = \frac{1}{1+e^{-(b_0+b_1 X_1+b_2 X_2+...))}}$$

where $b_0$ is a constant, $b_1,b_2, ...$ are coefficients multiplied by their respective variables, $X_1, X_2, ...$ are the features in the dataset (AKA columns or variables).

<b> I have to admit, formulas make me anxious too. But don't worry, no manual calculations will be performed, python will take care of all that. But it's good to know the basics</b>



## About the Data Used
In this notebook, I will follow the analysis mentioned in the paper [Machine learning in medicine: a practical introduction](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6425557/). The code in that paper is written in R. I translated it to python and modified it a little for the purposes of this course.

We will use for this course the dataset provided in the same paper mentioned above. This dataset was originally publicly available from the paper [Machine Learning Repository of University of California Irvine](http://archive.ics.uci.edu/).

This dataset [consists of characteristics, or features, of cell nuclei taken from breast masses which were sampled using fine-needle aspiration (FNA), a common diagnostic procedure in oncology. The clinical samples used to form this dataset were collected from January 1989 to November 1991.] (Sidey-Gibbons and Sidey-Gibbons 2019) Each sample is then classified as malignant or benign in the 'class' column (1= malignant and 0 = benign).



---
## Data Analysis Steps:
### 1. Import libraries and define functions
These are the functions that you will need for your following analyses. <b>You don't need to worry about this part</b> ... just run the cell below and move to the next :)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, recall_score,confusion_matrix
import statsmodels.api as sm
import seaborn as sns

def choose_model(model, c_type = 'best', plot_result = True):
    """
    This function chooses a model from a set of models identified using LogisticRegressionCV
    It can return the best model (model.C_) or the most parsimonious model, which is the model whose score is
    within 1 standard error from the best score
    :param model: logistic regression model with cross validation (LogisticRegressionCV)
    :param c_type: type of C value to return,
                   can be either 'best' for best model, or 'par' for the most parsimonious model
    :param plot_result: whether to plot the reult and show the best model and parsimonious model on the same figure
    :return: C value for the chosen model
    """
    n_folds = model.coefs_paths_[1.0].shape[0]
    c_vals = model.Cs_
    best_c = model.C_

    best_c_ind = np.where(np.abs(c_vals - model.C_) < 1e-10)[0][0]

    included_vars = np.sum(model.coefs_paths_[1.0].mean(axis=0) != 0,  axis=1) - 1  # the -1 is make sure the intercept is not included
    included_vars = included_vars[
        [int(item) for item in np.linspace(0, len(included_vars) - 1, 30)]]  # Take only 30 samples from included_vars
    scores = model.scores_[1.0].mean(axis=0)
    scores_sem = model.scores_[1.0].std(axis=0) / np.sqrt(n_folds)

    # Get 1 standard error of the mean (SEM) from the best accuracy,
    best_sem = scores_sem[best_c_ind]

    # finds the last point where scores are within one SEM from best score
    c1se_ind = np.where(scores[best_c_ind] - scores[0:best_c_ind] < best_sem)[0][0]
    c1se = model.Cs_[c1se_ind]  # least acceptable score
    if plot_result:
        fig, ax = plt.subplots(figsize=(8, 5))
        ax3 = ax.twiny()
        ax3.set_xticks(np.arange(0, len(included_vars) + 2), [''] + list(included_vars) + [''], font = 'Cambria', fontsize = 12)
        ax3.tick_params(width=0, length = 0)
        ax3.set_xlabel('Included Variables', font = "Cambria", fontsize = 18)
        ax.axvline(x=np.log(best_c), color='grey', ls='-', lw=1, label='Best Score Model')
        ax.axvline(x=np.log(c1se), color='grey', ls='-.', lw=1, label='Parsimonious Model')
        ax.errorbar(np.log(model.Cs_), scores, scores_sem, fmt='o', linewidth=1,
                    color='grey', mfc='royalblue', mec='none', capsize=4)
        ax.legend()
        x_axis_text = np.round(ax.get_xticks()[1:-1],1)
        y_axis_text = np.round(ax.get_yticks()[1:-1],1)
        ax.set_xticks(ticks =x_axis_text, labels =  x_axis_text,font = 'Cambria', fontsize = 12)
        ax.set_yticks(ticks =y_axis_text, labels =  y_axis_text,font = 'Cambria', fontsize = 12)
        ax.set_xlabel('log(C)',font = 'Cambria', fontsize = 18)
        ax.set_ylabel('Accuracy',font = 'Cambria', fontsize = 18)

    if c_type=='best':
        return model.C_[0]
    elif c_type == 'par':
        return c1se
    else:
        raise Warning("c_type can only be set to 'best' or 'par'")


def model_performance(model,X_test,y_test):
    """
    Print the coefficients and compute accuracy
    :param model: the model to be tested
    :return: print the coeffcieints and compute accuracy
    """
    # let's have a look at the coefficients and see if anything was removed
    print('Coefficients:')
    coefs = [model.intercept_[0]] + list(model.coef_[0])
    coefs = [str(np.round(item,3)) if item!=0 else "-" for item in coefs]
    coef_names = ['intercept'] + list(data.columns[1:-1])
    coefficients = pd.DataFrame(coefs, index=coef_names,columns=['value'])
    print(coefficients)
    print('\n Scores:')

    y_pred = model.predict(X_test)
    tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()
    sens = tp / (tp + fn)
    spec = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    return pd.DataFrame([[sens, spec, accuracy]], columns=['Sensitivity', 'Specificity', 'Accuracy'])


### 
### 2. Load the data



In [ ]:
data_dir = 'https://github.com/abdulrahman1123/analysis_examples/raw/main/breast_cancer_wisconsin.csv'
data = pd.read_csv(data_dir)
data = data.iloc[0:200,:]

data

##### 2.1. Have a look at the 'class' variable and plot missing values

In [ ]:
fig,(ax1,ax2) = plt.subplots(nrows = 1, ncols =2, figsize = (9,4))
sns.set_theme(style="darkgrid")
sns.countplot(x='class', data= data,ax = ax1)
ax1.set_title("Count of Cases in the Class Variable")

sns.heatmap(data.isnull(), cbar=False,ax= ax2)
ax2.set_title("Count of missing cases in all variables")

plt.show()

### 
### 3. Create training and test datasets

The following generally applies for all machine learning algorithms:

The first step to do is to <b>split your data into training and testing datasets</b>. The training dataset will be used to train the model and evaluate parameters, while the testing dataset will only be used at the end to test how well did the model learn.

The training dataset can be further divided into training and validation datasets, where the validation dataset is used to evaluate the parameters in order to reach to the best model. The way we will be doing this is represented in the following figure from [scikit-learn website](https://scikit-learn.org/stable/_images/grid_search_cross_validation.png)

<img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" width=440 height=305 />

After splitting the data, we will impute the missing values using the mean of each column. This will be done separately for training and testing datasets.

Then, the data will be normalized (mean = 0 and sd = 1) so that the model can coverge faster (i.e. reach the best solution faster)

In [ ]:
# Choose features
X = data.iloc[:,1:-1]
y = data.iloc[:,-1]

# Divide into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=5)

print(f"Size of training dataset = {X_train.shape} and testing dataset = {X_test.shape}")

# Using the mean, impute the training and testing datasets separately
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
X_train = imp.fit_transform(X_train.copy())
X_test = imp.transform(X_test.copy())


# scale both X_train and X_test for faster convergence
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


### 
### 4. Create and fit the model

To create the best model, we want to use L1-regularization. This is basically a penalty that is applied to the model to remove irrelevant features.

L1-regularization basically assigns values for each of the variables to determine how sensitive the final model is to each of the variables. This regularization can be made more or less sensitive by modifying its parameter (C). Lower C values mean more sensitivity -> higher chance for variables to be removed. Higher C values mean less sensitivity and so on.

The aim of using regularization at all is to remove irrelevant variables and prevent overfitting

For further information, you can have a look at this [website](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c) for simplified explanation of L1 regularization.

First, let's choose a number of c-values for the model to validate during training. The model will move over the values you provided to find the best value of C for the training dataset. <i>(Note: If you do not provide a list of Cs, it will choose 10 values between 1e-4 and 1e4 by default).</i>


In [ ]:
# Choose a number of c-values for the model to test.
c_vals = np.logspace(-3,1, 50) # The smaller the value, the higher the penalty is
print (f"There are {len(c_vals)} C-values. They range from {c_vals[0]} to {c_vals[-1]}")

cv_model = LogisticRegressionCV(Cs=c_vals,penalty='l1', cv=10, tol=0.001, solver='saga', scoring='accuracy')
# you can also choose scoring = 'neg_mean_squared_error', but remember to multiply scores with -1


print ("\nSearching for best model ...\n")
# fit the model to the training data set
cv_model.fit(X_train,y_train)

print (f"Best model was found. Best C value is {cv_model.C_}")

### 
### 5. Show the results
##### 5.1 For the best model
A Table of coefficients will be displayed along with model performance on testing dataset

In [ ]:
model_performance(cv_model,  X_test, y_test)

##### 5.2 For the most parsimonious model
A Parsimonious model can be chosen by chosing a smaller C value. This C value should have accuracy that lies within one standard error of the mean (SEM) from the best accuracy achieved. Basically, you are choosing a C with very similar performance, yet with less number of variables included. This follows what is used in glmnet package for R.

In [ ]:
# Choose parsimonious model
par_c = choose_model(model = cv_model,c_type='par', plot_result=True)
par_model = LogisticRegression(C=par_c, penalty='l1', tol = 0.001, solver = 'saga').fit(X_train,y_train)
model_performance(par_model, X_test, y_test)

# 
# Optional: Logistic Regression using statsmodels library
If your aim is not to predict future cases nor is it to comment on the generalizability of the model. In other words, if your aim is to measure the inferential significance of your model, in this case, you would want the traditional logistic regression, where z-values are computed and p-values assigned. This can be done using the statsmodels library, which provides more advanced computations to scikit learn, but it is less comprehensive in terms of the options it can provide.

We will apply logistic regression using statsmodels library on the same training dataset we used above. Please remember two things:

1- the penalty term here is called 'alpha' and is the inverse of the scikit-learn C-value

2- the solvers used to find the best model are slightly different. So, the values might differ a little bit, but should be in the same range.

In [ ]:
# First, add a constant to your data, because statsmodels does not do that by default
X_train_sm = sm.add_constant(X_train)
X_test_sm = sm.add_constant(X_test)

alpha = [0] + [1/cv_model.C_[0]]*X_train.shape[1] # This applies 1/C penalty to all variables except for the constant
logit_mod = sm.Logit(y_train, X_train_sm).fit_regularized(method='l1', alpha=alpha)


print(logit_mod.summary())

### 
### You can also compute model performance on the testing data set

In [ ]:
y_pred = logit_mod.predict(X_test_sm) # This gives probabilities of y
y_pred_cat = np.where(y_pred>0.5,1,0) # This converts probabilities to either 0 or 1
tn, fp, fn, tp = confusion_matrix(y_test, y_pred_cat).ravel()
sens = tp / (tp + fn)
spec = tn / (tn + fp)
accuracy = (tp + tn) / (tp + tn + fp + fn)
pd.DataFrame([[sens, spec, accuracy]], columns=['Sensitivity', 'Specificity', 'Accuracy'])

### 
### Awesome! We have the same results as the scikit-learn model above!
### 

---
### Further reading:
[Machine learning in medicine: a practical introduction](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC6425557/)

[Data Preparation for Machine Learning](https://books.google.de/books?id=uAPuDwAAQBAJ)

[Web page about regularization](https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c)

[Scikit-learn web page about LogisticRegressionCV function](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegressionCV.html)

[Statsmodels web page about logistic regression function with regularization](https://www.statsmodels.org/stable/generated/statsmodels.discrete.discrete_model.Logit.fit_regularized.html)